#利用Textrank做文本摘要的核心思想很简单，和著名的网页排名算法PageRank类似：每个句子可以作为一个网络中的节点（称为节点i），与之相连的其他节点（例如节点j）会对其重要度产生一定的“贡献值”，该“贡献值”与节点j自身的重要度以及i、j之间的相似度（也可以称为连接的强度）有关，只需要对整个图进行迭代直至收敛，最后各节点的分值即是该句子的重要性，根据重要性排序后选取前k个句子即可作为摘要。

In [17]:
import jieba
import re 

In [15]:
def split_sentences(full_text):
    sents = re.split(u'[\n。]', full_text)
    sents = [sent for sent in sents if len(sent) > 0]  # 去除只包含\n或空白符的句子
    return sents

# Textrank的原始论文中，句子相似度是基于两个句子的共现词的个数计算的，在此沿用论文的公式：
# Similarity(Si,Sj)={wk|wk∈Si&wk∈Sj}/(log(Si)+log(Sj))
# 实现时，采用共现词计数进行相似度计算，输入的是每个句子的terms，以避免重复分词。代码如下：

In [4]:
def cal_sim(wordlist1, wordlist2):
    """
    给定两个句子的词列表，计算句子相似度。计算公式参考Textrank论文
    :param wordlist1:
    :param wordlist2:
    :return:
    """
    co_occur_sum = 0
    wordset1 = list(set(wordlist1))
    wordset2 = list(set(wordlist2))
    for word in wordset1:
        if word in wordset2:
            co_occur_sum += 1.0
    if co_occur_sum < 1e-12:  # 防止出现0的情况
        return 0.0
    denominator = math.log(len(wordset1)) + math.log(len(wordset2))
    if abs(denominator) < 1e-12:
        return 0.0
    return co_occur_sum / denominator

#利用networkx库创建一个graph实例，调用networkx的pagerank方法对graph实例进行处理即可，需要注意的是，networkx有三种pagerank的实现，分别是pagerank、pagerank_numpy和pagerank_scipy，从名称可以看出来它们分别采用了不同的底层实现，此处我们任选其一即可。代码如下：

In [25]:
def text_rank(sentences, num=10, pagerank_config={'alpha': 0.85, }):
    """
    对输入的句子进行重要度排序
    :param sentences: 句子的list
    :param num: 希望输出的句子数
    :param pagerank_config: pagerank相关设置，默认设置阻尼系数为0.85
    :return:
    """
    sorted_sentences = []
    sentences_num = len(sentences)#sentences分句后的句子
    wordlist = []  # 存储wordlist避免重复分词，其中wordlist的顺序与sentences对应
    for sent in sentences:
        tmp = []
        cur_res = jieba.cut(sent)
        for i in cur_res:
            tmp.append(i)
        wordlist.append(tmp)#分词后的句子列表
    graph = np.zeros((sentences_num, sentences_num))
    for x in range(sentences_num):
        for y in range(x, sentences_num):
            similarity = cal_sim(wordlist[x], wordlist[y])
            graph[x, y] = similarity
            graph[y, x] = similarity
    nx_graph = nx.from_numpy_matrix(graph)#从numpy矩阵返回图形，numpy矩阵被解释为该图的邻接矩阵。
    scores = nx.pagerank(nx_graph, **pagerank_config)  # this is a dict 核心：PageRank根据传入链接的结构计算图G中节点的排名。它最初被设计为对网页进行排名的算法。
    sorted_scores = sorted(scores.items(), key=lambda item: item[1], reverse=True)#核心
    for index, score in sorted_scores:
        item = {"sent": sentences[index], 'score': score, 'index': index}
        sorted_sentences.append(item)
    return sorted_sentences[:num]#函数返回的结果中即包含了num句关键句子，可以作为组成摘要的基础。

# 由text_rank得到的前k个句子，只能表示这几个句子很重要，然而他们在逻辑上很难串联起来。如何重组织摘要，在学术界也是一大研究热点。根据不同的处理粒度（句子级、字词级）和不同的处理思路（根据语义重组还是改变现有词句的顺序），生成的摘要在阅读性上有很大的不同。
# 在此为了简便，选取最简单的，根据句子在文章中出现的顺序对text_rank结果进行重排序。代码如下：

In [6]:
def extract_abstracts(full_text, sent_num=10):
    """
    摘要提取的入口函数，并根据textrank结果进行摘要组织
    :param full_text:
    :param sent_num:
    :return:
    """
    sents = split_sentences(full_text)
    trank_res = text_rank(sents, num=sent_num)
    sorted_res = sorted(trank_res, key=lambda x: x['index'], reverse=False)
    return sorted_res

In [3]:
# 测试

In [22]:
import codecs
raw_text = codecs.open('./text.txt', 'r', 'utf8').read()

In [28]:
import math
import networkx as nx
res = extract_abstracts(raw_text, sent_num=5)#只是找出关键句子，组成摘要的基础。

In [31]:
for s in res:
    print(s['score'], s['sent'])

0.06619861502317748 ﻿传了两个月的中国公司要收购 AC 米兰的事情终于有了一个确切的消息，拥有 AC 米兰俱乐部股权的 Fininvest 公司官方正式确认正在和一家来自中国的企业商谈俱乐部股权出售事宜
0.062095949005264744 2015 年 11 月，AC 米兰老板贝卢斯科尼访华，并称就美丽之冠绿卡收购 AC 米兰一定数量的股权一事达成了合作意向，然而这件事也就此没了下文
0.06128367799191754 一年多前，曾有泰国财团为 AC 米兰开出了 5 亿欧元收购 48% 的股份的价码，这意味着当时 AC 米兰的估值为 10 亿欧元
0.06107030979584044 根据 AC 米兰官网上的数据，这家俱乐部从 2007 年开始就一直处在净亏损的状态中，2014 年的亏损额接近 1 亿欧元，更是创下了历史新高
0.062467472820868564 如果 AC 米兰真的被中国人买下来了，那么在这个买家看来，他买下来的也绝不仅仅只是一个足球俱乐部而已
